In [ ]:
# Packages to be imported

from ipynb.fs.full.Functions import *

In [ ]:
time_step =[]
Mbits_transmitted = []


# Data for training.
with open('./Traffic_Train_Data.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter = ',')
    next(reader)
    for row in reader:
        Mbits_transmitted.append(float(row[1]))
        time_step.append(int(row[0]))

#converting the lists into arrays
series_trans = np.array(Mbits_transmitted)
time = np.array(time_step)


window_size = 20
batch_size = 16
comms_round = 10
Clients = create_clients(series_trans,35, 'clients')

In [ ]:
#initialize global model
smlp_global = SimpleMLP()
global_model = smlp_global.build()
       
#commence global training loop
for comm_round in range(comms_round):
    optimizer1 = tf.keras.optimizers.SGD(learning_rate=3e-4)        
    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()
   
    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(Clients)
    random.shuffle(client_names)
   
    #loop through each client and create new local model
    for client in client_names:
        window_size = 20
        batch_size = 16
        dataset = windowed_dataset(np.array(Clients[client]), window_size, batch_size, len(Clients[client]))        
        smlp_local = SimpleMLP()
        local_model = smlp_local.build()
        local_model.set_weights(global_weights)
        #compiling the model with a mean squared error loss and a stochastic gradient descent optimizer
        optimizer = tf.keras.optimizers.SGD(learning_rate=3e-4)
        loss_fn = tf.keras.losses.MeanSquaredError()
        for step, (x,y) in enumerate(dataset):
            with tf.GradientTape() as tape:
                y_hat = local_model(x)
                loss = loss_fn(y, y_hat)
                gradients = tape.gradient(loss, local_model.trainable_weights)
            optimizer.apply_gradients(zip(gradients,local_model.trainable_weights))
        print(client, loss)
        print("\n\n\n")
        print(len(gradients))
        gradient = gradients
        print("\n\n\n")
        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(Clients, client)
        scaled_gradient = scaling_factor*np.array(gradient)
        scaled_local_weight_list.append(scaled_gradient)
       
        #clear session to free memory after each communication round
        K.clear_session()

    print(comm_round)

    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_gradient = sum(scaled_local_weight_list)
    #update global model
    print(len(average_gradient))
    optimizer1.apply_gradients(zip(average_gradient,global_model.trainable_weights))
  

In [ ]:
# For testing the performance of the model.
with open('./Traffic_Test_Data.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter = ',')
    next(reader)
    for row in reader:
        Mbits_transmitted.append(float(row[1]))
        time_step.append(int(row[0]))

#converting the lists into arrays
series_trans_test = np.array(Mbits_transmitted)
print(get_average_mse(series_trans_test, global_model, window_size, 10))